In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import joblib
from copy import deepcopy
import traing_function_real_data as tp
import build_new_data as bnd
# import assortment as at
import warnings
import torch
from tqdm import tqdm
from sklearn.model_selection import cross_val_score

## Split the original data into training set and test set

In [3]:
data = pd.read_csv('df_OR_normal_process_with_nonpurchase_clean.csv')
data = data.drop(columns=['prop_id', 'srch_length_of_stay'])

In [ ]:
print(max(data['prop_location_score1']), min(data['prop_location_score1']))
print(max(data['prop_location_score2']), min(data['prop_location_score2']))
print(max(data['prop_log_historical_price']), min(data['prop_log_historical_price']))
print(max(data['price_usd']), min(data['price_usd']))
print(max(data['srch_adults_count']), min(data['srch_adults_count']))
print(max(data['srch_children_count']), min(data['srch_children_count']))
print(max(data['srch_booking_window']), min(data['srch_booking_window']))

In [3]:
conti_columns = ['prop_location_score1', 'prop_location_score2', 
                    'prop_log_historical_price', 'price_usd', 'srch_adults_count', 'srch_children_count']
for column in conti_columns:
  max1 = max(data[column])
  min1 = min(data[column])
  data[column] = (data[column]-min1)/(max1-min1)

In [ ]:
cat_columns = ['prop_starrating', 'prop_review_score', 'prop_brand_bool','promotion_flag','srch_saturday_night_bool','srch_booking_window']
feature_sizes = []
for cat in cat_columns:
    temp_list = list(data.groupby(cat).groups)
    temp_dic = {}
    for i in range(len(temp_list)):
        temp_dic[temp_list[i]] = i
    data[cat] = data[cat].transform(lambda s: temp_dic[s])
    print(cat, len(temp_list))
    feature_sizes.append(len(temp_list))
feature_sizes

In [5]:
total_order_list = list(set(data['srch_id']))
total_num = len(total_order_list)

total_dic = {}
for rows in range(len(data)):
    temp_id = data.loc[rows,'srch_id']
    if temp_id in total_dic:
        total_dic[temp_id].append(rows)
    else:
        total_dic[temp_id] = [rows]

In [6]:
order_list_random = np.random.permutation(total_order_list)
train_order = order_list_random[:int(0.8*total_num)]
valid_order = order_list_random[int(0.8*total_num):int(0.9*total_num)]
test_order = order_list_random[int(0.9*total_num):]

In [7]:
train_list = []
valid_list = []
test_list = []
for orderid in train_order:
    train_list.extend(total_dic[orderid])
for orderid in valid_order:
    valid_list.extend(total_dic[orderid])
for orderid in test_order:
    test_list.extend(total_dic[orderid])
train_data = data.loc[train_list]
train_data = train_data.reset_index(drop=True)
valid_data = data.loc[valid_list]
valid_data = valid_data.reset_index(drop=True)
test_data = data.loc[test_list]
test_data = test_data.reset_index(drop=True)

## Record the assortment information

In [8]:
train_dic = {}
for rows in range(len(train_data)):
    temp_id = train_data.loc[rows,'srch_id']
    if temp_id in train_dic:
        train_dic[temp_id].append(rows)
    else:
        train_dic[temp_id] = [rows]

valid_dic = {}
temp_id = -1
for rows in range(len(valid_data)):
    temp_id = valid_data.loc[rows,'srch_id']
    if temp_id in valid_dic:
        valid_dic[temp_id].append(rows)
    else:
        valid_dic[temp_id] = [rows]

test_dic = {}
temp_id = -1
for rows in range(len(test_data)):
    temp_id = test_data.loc[rows,'srch_id']
    if temp_id in test_dic:
        test_dic[temp_id].append(rows)
    else:
        test_dic[temp_id] = [rows]
np.save('train_data_for_simulator\\train_dic.npy', train_dic)
np.save('train_data_for_simulator\\valid_dic.npy', valid_dic)
np.save('train_data_for_simulator\\test_dic.npy', test_dic)

In [9]:
# warnings.filterwarnings('ignore')

label = train_data['booking_bool']
del_feature = ['srch_id','booking_bool']
features = [i for i in train_data.columns if i not in del_feature]

data_np = np.array(train_data[features])
data_np_cate = np.zeros(shape = data_np.shape)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np_cate[:,idx] = data_np[:,idx]
        data_np[:,idx] = 1

for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item != 1:
        data_np[:,idx] = 1

total_train_data = np.array([data_np_cate,data_np])

In [10]:
np.save('train_data_for_simulator\data_ztn.npy',total_train_data)
np.save('train_data_for_simulator\label_ztn.npy',label)

In [11]:
label = valid_data['booking_bool']
del_feature = ['srch_id','booking_bool']
features = [i for i in valid_data.columns if i not in del_feature]
data_np = np.array(valid_data[features])
data_np_cate = np.zeros(shape = data_np.shape)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np_cate[:,idx] = data_np[:,idx]
        data_np[:,idx] = 1
        
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item != 1:
        data_np[:,idx] = 1

total_valid_data = np.array([data_np_cate,data_np])
total_valid_data.shape

(2, 105525, 12)

In [12]:
np.save('train_data_for_simulator\\valid_data_ztn.npy',total_valid_data)
np.save('train_data_for_simulator\\valid_label_ztn.npy',label)

In [13]:
label = test_data['booking_bool']
del_feature = ['srch_id','booking_bool']
features = [i for i in test_data.columns if i not in del_feature]
data_np = np.array(test_data[features])
data_np_cate = np.zeros(shape = data_np.shape)
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item !=1 :
        data_np_cate[:,idx] = data_np[:,idx]
        data_np[:,idx] = 1
        
for idx,item in enumerate([6, 10, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2]):
    if item != 1:
        data_np[:,idx] = 1

total_test_data = np.array([data_np_cate,data_np])

In [14]:
np.save('train_data_for_simulator\\test_data_ztn.npy',total_test_data)
np.save('train_data_for_simulator\\test_label_ztn.npy',label)

## load data

In [2]:
train_data = np.load('train_data_for_simulator\\data_ztn.npy')
train_label = np.load('train_data_for_simulator\\label_ztn.npy')
train_dic = np.load('train_data_for_simulator\\train_dic.npy', allow_pickle=True).item()

valid_data = np.load('train_data_for_simulator\\valid_data_ztn.npy')
valid_label = np.load('train_data_for_simulator\\valid_label_ztn.npy')
valid_dic = np.load('train_data_for_simulator\\valid_dic.npy', allow_pickle=True).item()

test_data = np.load('train_data_for_simulator\\test_data_ztn.npy')
test_label = np.load('train_data_for_simulator\\test_label_ztn.npy')
test_dic = np.load('train_data_for_simulator\\test_dic.npy', allow_pickle=True).item()

In [3]:
data = deepcopy(train_data)
train_data_assort = np.array([np.concatenate([data[0],np.zeros((841500, 6)),np.zeros((841500, 4))],axis = 1),np.concatenate([data[1],np.ones((841500, 6)),np.ones((841500, 4))],axis = 1)])
for i in train_dic.values():
    data_temp = np.zeros((len(i), 6))
    data_cate_temp = np.zeros((len(i), 4))
    min1 = np.min(data[1,i,5])
    max1 = np.max(data[1,i,5])
    mean1 = np.mean(data[1,i,5])
    min2 = np.min(data[1,i,6])
    max2 = np.max(data[1,i,6])
    mean2 = np.mean(data[1,i,6])
    data_temp[:,0] = min1
    data_temp[:,1] = max1
    data_temp[:,2] = mean1
    data_temp[:,3] = min2
    data_temp[:,4] = max2
    data_temp[:,5] = mean2
    for idx,j in enumerate(i):
        if data[1,j,5] == np.min(data[1,i,5]):
            data_cate_temp[idx,0] = 1
        if data[1,j,5] == np.max(data[1,i,5]):
            data_cate_temp[idx,1] = 1
        
        if data[1,j,6] == np.min(data[1,i,6]):
            data_cate_temp[idx,2] = 1
        if data[1,j,6] == np.max(data[1,i,6]):
            data_cate_temp[idx,3] = 1
    train_data_assort[0,i,18:] = data_cate_temp
    train_data_assort[1,i,12:18] = data_temp

In [4]:
data = deepcopy(valid_data)
valid_data_assort = np.array([np.concatenate([data[0],np.zeros((data.shape[1], 6)),np.zeros((data.shape[1], 4))],axis = 1),np.concatenate([data[1],np.ones((data.shape[1], 6)),np.ones((data.shape[1], 4))],axis = 1)])
for i in valid_dic.values():
    data_temp = np.zeros((len(i), 6))
    data_cate_temp = np.zeros((len(i), 4))
    min1 = np.min(data[1,i,5])
    max1 = np.max(data[1,i,5])
    mean1 = np.mean(data[1,i,5])
    min2 = np.min(data[1,i,6])
    max2 = np.max(data[1,i,6])
    mean2 = np.mean(data[1,i,6])
    data_temp[:,0] = min1
    data_temp[:,1] = max1
    data_temp[:,2] = mean1
    data_temp[:,3] = min2
    data_temp[:,4] = max2
    data_temp[:,5] = mean2
    for idx,j in enumerate(i):
        if data[1,j,5] == np.min(data[1,i,5]):
            data_cate_temp[idx,0] = 1
        if data[1,j,5] == np.max(data[1,i,5]):
            data_cate_temp[idx,1] = 1
        
        if data[1,j,6] == np.min(data[1,i,6]):
            data_cate_temp[idx,2] = 1
        if data[1,j,6] == np.max(data[1,i,6]):
            data_cate_temp[idx,3] = 1
    valid_data_assort[0,i,18:] = data_cate_temp
    valid_data_assort[1,i,12:18] = data_temp

In [5]:
data = deepcopy(test_data)
test_data_assort = np.array([np.concatenate([data[0],np.zeros((data.shape[1], 6)),np.zeros((data.shape[1], 4))],axis = 1),np.concatenate([data[1],np.ones((data.shape[1], 6)),np.ones((data.shape[1], 4))],axis = 1)])
for i in test_dic.values():
    data_temp = np.zeros((len(i), 6))
    data_cate_temp = np.zeros((len(i), 4))
    min1 = np.min(data[1,i,5])
    max1 = np.max(data[1,i,5])
    mean1 = np.mean(data[1,i,5])
    min2 = np.min(data[1,i,6])
    max2 = np.max(data[1,i,6])
    mean2 = np.mean(data[1,i,6])
    data_temp[:,0] = min1
    data_temp[:,1] = max1
    data_temp[:,2] = mean1
    data_temp[:,3] = min2
    data_temp[:,4] = max2
    data_temp[:,5] = mean2
    for idx,j in enumerate(i):
        if data[1,j,5] == np.min(data[1,i,5]):
            data_cate_temp[idx,0] = 1
        if data[1,j,5] == np.max(data[1,i,5]):
            data_cate_temp[idx,1] = 1
        
        if data[1,j,6] == np.min(data[1,i,6]):
            data_cate_temp[idx,2] = 1
        if data[1,j,6] == np.max(data[1,i,6]):
            data_cate_temp[idx,3] = 1
    test_data_assort[0,i,18:] = data_cate_temp
    test_data_assort[1,i,12:18] = data_temp

## 1 MNL

### 1.1 Traing the MNL model

In [3]:
train_data_MNL = np.concatenate((train_data,valid_data),axis = 1)
train_label_MNL = np.concatenate((train_label,valid_label),axis = 0)
train_dic_MNL = deepcopy(train_dic)
for i in valid_dic.keys():
    if i in train_dic.keys():
        print('error')
    else:
        train_dic_MNL[i] = list(np.array(valid_dic[i]) + train_data.shape[1])

In [ ]:
PATH = "trained_model_ztn\\MNL_parameters_ztn.pt"
model_MNL = tp.train_data(train_data_MNL,train_label_MNL, train_dic_MNL, NUM_EPOCHS = 100,BATCH_SIZE = 32,path = PATH)

### 1.2 Prediction Results

In [7]:
model_MNL = torch.load("trained_model_ztn\\MNL_parameters_ztn.pt")
X = torch.from_numpy(test_data[0]).to(torch.long)
weight = torch.from_numpy(test_data[1]).to(torch.float)
utility_MNL = model_MNL([X,weight])
utility_MNL = utility_MNL.detach().numpy()
#u = float(u.detach().numpy())

In [8]:
for item in test_dic:
    temp_lst = test_dic[item]
    temp_sum = np.sum(np.exp(np.array(utility_MNL[test_dic[item]]))) 
    for i in temp_lst:
        utility_MNL[i] = np.exp(utility_MNL[i]) /temp_sum

total = 0
count = 0
top_n = 3
rmse = []
denominator = 0
for item in test_dic:
    denominator += len(test_dic[item])
    rmse.append(np.sum((utility_MNL[test_dic[item]]-test_label[test_dic[item]])**2))
    temp_prob_max = np.argsort(-utility_MNL[test_dic[item]])[0:top_n]
    temp_test_y = list(test_label[test_dic[item]])
    total += 1
    temp_y_max = temp_test_y.index(max(temp_test_y))
    if temp_y_max in temp_prob_max:
        count += 1
print("RMSE: ",np.sqrt(sum(rmse)/denominator))
print(count/total)

## 2 MMNL

### 2.1 Train the model

In [3]:
train_data_MNL = np.concatenate((train_data,valid_data),axis = 1)
train_label_MNL = np.concatenate((train_label,valid_label),axis = 0)
train_dic_MNL = deepcopy(train_dic)
for i in valid_dic.keys():
    if i in train_dic.keys():
        print('error')
    else:
        train_dic_MNL[i] = list(np.array(valid_dic[i]) + train_data.shape[1])

In [ ]:
#PATH = "trained_model_ztn\\MMNL_parameters_ztn.pt"
model_MMNL_list,alpha = tp.CG_algo(train_data_MNL,train_label_MNL, train_dic_MNL, NUM_EPOCHS = 100,BATCH_SIZE = 32)

### 2.2 Prediction Results

In [12]:
model_MMNL_list = np.load("trained_model_ztn\\MMNL.npy",allow_pickle=True)
alpha = np.load("trained_model_ztn\\alpha.npy")

In [13]:
pro_temp = []
for model_MNL in model_MMNL_list:
    X = torch.from_numpy(test_data[0]).to(torch.long)
    weight = torch.from_numpy(test_data[1]).to(torch.float)
    utility_MNL = model_MNL([X,weight])
    utility_MNL = utility_MNL.detach().numpy()
    for item in test_dic:
        temp_lst = test_dic[item]
        temp_sum = np.sum(np.exp(np.array(utility_MNL[test_dic[item]]))) 
        for i in temp_lst:
            utility_MNL[i] = np.exp(utility_MNL[i]) /temp_sum
    pro_temp.append(utility_MNL)
pro = 0
for i in range(len(pro_temp)):
    pro = alpha[i]*np.array(pro_temp[i]) + pro

In [14]:
total = 0
count = 0
top_n = 3
rmse = []
denominator = 0
for item in test_dic:
    denominator += len(test_dic[item])
    rmse.append(np.sum((pro[test_dic[item]]-test_label[test_dic[item]])**2))
    temp_prob_max = np.argsort(-pro[test_dic[item]])[0:top_n]
    temp_test_y = list(test_label[test_dic[item]])
    total += 1
    temp_y_max = temp_test_y.index(max(temp_test_y))
    if temp_y_max in temp_prob_max:
        count += 1
print("RMSE: ",np.sqrt(sum(rmse)/denominator))
print(count/total)

RMSE:  0.19198633144423044
0.29910821884791516


## 3 DeepFM

### 3.1 Train the model

In [ ]:
PATH = "trained_model_ztn\\DeepFM_parameters_ztn.pt"
model_DeepFM = tp.train_DeepFM(train_data,train_label,valid_data,valid_label,valid_dic, NUM_EPOCHS = 100,BATCH_SIZE = 256,LR = 0.05,weight1 = 31,path = PATH)

### 3.2 Prediction Results

In [16]:
model_DeepFM = torch.load("trained_model_ztn\\DeepFM_parameters_ztn.pt")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cat = torch.from_numpy(test_data[0]).to(torch.long).to(device)
weight = torch.from_numpy(test_data[1]).to(torch.float).to(device)
outputs = model_DeepFM([cat, weight])
outputs = outputs.cpu().detach().numpy()
pro = np.exp(outputs)/(np.exp(outputs)+1)

In [18]:
total = 0
count = 0
top_n = 3
rmse = []
denominator = 0
for item in test_dic:
    denominator += len(test_dic[item])
    temp_pro = pro[test_dic[item]]/np.sum(pro[test_dic[item]])
    rmse.append(np.sum((temp_pro-test_label[test_dic[item]])**2))
    temp_prob_max = np.argsort(-pro[test_dic[item]])[0:top_n]
    temp_test_y = list(test_label[test_dic[item]])
    total += 1
    temp_y_max = temp_test_y.index(max(temp_test_y))
    if temp_y_max in temp_prob_max:
        count += 1
print("RMSE: ",np.sqrt(sum(rmse)/denominator))
print(count/total)

RMSE:  0.1922712930116513
0.35936370209689084


## 4 DeepFM-a

### 4.1 Train the model

In [ ]:
PATH = "trained_model_ztn\\DeepFM-a_parameters_ztn.pt"
model_DeepFM = tp.train_DeepFMa(train_data_assort,train_label,valid_data_assort,valid_label,valid_dic, NUM_EPOCHS = 100,BATCH_SIZE = 256,LR = 0.05,weight1 = 31,path = PATH)

### 4.2 Prediction Results

In [19]:
model_DeepFM = torch.load("trained_model_ztn\\DeepFM-a_parameters_ztn.pt")

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cat = torch.from_numpy(test_data_assort[0]).to(torch.long).to(device)
weight = torch.from_numpy(test_data_assort[1]).to(torch.float).to(device)
outputs = model_DeepFM([cat, weight])
outputs = outputs.cpu().detach().numpy()
pro = np.exp(outputs)/(np.exp(outputs)+1)

In [22]:
total = 0
count = 0
top_n = 3
rmse = []
denominator = 0
for item in test_dic:
    denominator += len(test_dic[item])
    temp_pro = pro[test_dic[item]]/np.sum(pro[test_dic[item]])
    rmse.append(np.sum((temp_pro-test_label[test_dic[item]])**2))
    temp_prob_max = np.argsort(-pro[test_dic[item]])[0:top_n]
    temp_test_y = list(test_label[test_dic[item]])
    total += 1
    temp_y_max = temp_test_y.index(max(temp_test_y))
    if temp_y_max in temp_prob_max:
        count += 1
print("RMSE: ",np.sqrt(sum(rmse)/denominator))
print(count/total)

RMSE:  0.19214619268848493
0.34683056158110387


## 5 Exponential-Based 预测

### 5.1 Train the model

In [8]:
train_data_exp = np.concatenate((train_data,valid_data),axis = 1)
train_label_exp = np.concatenate((train_label,valid_label),axis = 0)
train_dic_exp = deepcopy(train_dic)
for i in valid_dic.keys():
    if i in train_dic.keys():
        print('error')
    else:
        train_dic_exp[i] = list(np.array(valid_dic[i]) + train_data.shape[1])

In [ ]:
PATH = "trained_model_ztn\\EXP_parameters_ztn.pt"
model_MNL = tp.train_data(train_data_exp,train_label_exp, train_dic_exp, NUM_EPOCHS = 100,BATCH_SIZE = 32,path = PATH, model_type = 'EXP')

### 5.2 Prediction Results

In [12]:
PATH = "trained_model_ztn\\EXP_parameters_ztn.pt"
model_exp = torch.load(PATH)
X = torch.from_numpy(test_data[0]).to(torch.long)
weight = torch.from_numpy(test_data[1]).to(torch.float)
utility_exp = model_exp([X,weight])
utility_exp = utility_exp.detach().numpy()

In [13]:
pro_exp = np.zeros_like(utility_exp)
for item in test_dic:
    temp_lst = test_dic[item]
    utility_temp = utility_exp[test_dic[item]]
    utility_temp = np.sort(utility_temp)
    for i in temp_lst:
        index = np.where(utility_temp == utility_exp[i])[0][0]
        Q1 = np.exp(-np.sum(utility_temp[index:] - utility_temp[index]))/(utility_temp.shape[0]-index)
        if index == 0:
            pro_exp[i] = Q1
        else:
            Q2 = 0
            for j in range(index):
                Q2 += np.exp(-np.sum(utility_temp[j:] - utility_temp[j]))/((utility_temp.shape[0]-j-1)*(utility_temp.shape[0]-j))
            pro_exp[i] = Q1-Q2

In [14]:
total = 0
count = 0
top_n = 3
rmse = []
denominator = 0
for item in test_dic:
    denominator += len(test_dic[item])
    rmse.append(np.sum((pro_exp[test_dic[item]]-test_label[test_dic[item]])**2))
    temp_prob_max = np.argsort(-pro_exp[test_dic[item]])[0:top_n]
    temp_test_y = list(test_label[test_dic[item]])
    total += 1
    temp_y_max = temp_test_y.index(max(temp_test_y))
    if temp_y_max in temp_prob_max:
        count += 1
print("RMSE: ",np.sqrt(sum(rmse)/denominator))
print(count/total)

RMSE:  0.19186398916432948
0.3234514340805013
